In [123]:
import simpy
import random
import math
import numpy

In [124]:
RANDOM_SEED = 978
INTERARRIVAL_MEAN = 6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

CALL_MEAN = 5 
CALL_RATE = 1.0 / CALL_MEAN


SERVICE_TIME_OPERATOR1_MEAN = 12
SERVICE_TIME_OPERATOR_VARIENCE = 36
mu= math.log(SERVICE_TIME_OPERATOR1_MEAN**2/math.sqrt(SERVICE_TIME_OPERATOR_VARIENCE+SERVICE_TIME_OPERATOR1_MEAN**2))
sigma=math.sqrt(math.log(1+SERVICE_TIME_OPERATOR_VARIENCE /SERVICE_TIME_OPERATOR1_MEAN))


SERVICE_TIME_OPERATOR2_MEAN = 4
SERVICE_TIME_OPERATOR2_DEVIATION = math.sqrt(3)

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60
CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

In [125]:
# Duration of the conversation between the customer and the operator (Operator1 service time)
service_times_operator1 = []

# Duration of the conversation between the customer and the operator2 (operator2 service time)
service_times_operator2 = []

# Time spent by a customer while it waits for the operator1 (Operator1 Queue waiting time)
queue_w_times_operator1 = []

# Time spent by a customer while it waits for the operator2 (operator2 Queue waiting time)
queue_w_times_operator2 = []

# The number of customer in the answering system
queue_w_customer = []


# Simulation end time
end_time = 0;



In [126]:
class Customer(object):
    def __init__(self, name, env, operator1, operator2):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        queue_w_customer.append(self.env.now - self.arrival_t)
        global end_time 
        prob=random.uniform(0,10)
        print(prob)
        error=random.uniform(0,10)
        print(error)
        
        yield env.timeout(random.expovariate(CALL_RATE))
        if len(queue_w_customer)<101:
            if error>1 :
                if prob<3:    
                    yield self.env.process(self.call1())
                if prob>=3:    
                    yield self.env.process(self.call2())
            else:
                 print('%s A caller hangs up at %g' % (self.name, self.env.now))
        else:
             print('%s A caller drops  without answering. at %g' % (self.name, self.env.now))
             print(len(queue_w_customer)) 
             queue_w_customer.pop()   


            
    def call1(self):
        
        with operator1.request() as req:
            yield req
            print('%s is assigned to a operator1 at %g' % (self.name, self.env.now))
            queue_w_times_operator1.append(self.env.now - self.arrival_t)
            yield self.env.process(self.get_help1_operator1())
            print('%s is done operator1 at %g' % (self.name, self.env.now))
            self.end_time = self.env.now
            queue_w_customer.pop()
            

    def call2(self):
        
        with operator2.request() as req:
            yield req
            print('%s is assigned to a operator2 at %g' % (self.name, self.env.now))
            queue_w_times_operator2.append(self.env.now - self.arrival_t)
            yield self.env.process(self.get_help2_operator2())
            print('%s is done operator2 at %g' % (self.name, self.env.now))
            self.end_time = self.env.now
            queue_w_customer.pop()

            

    def get_help1_operator1(self):
        duration = numpy.random.lognormal(mu, sigma)
        while duration < 0:
            duration = numpy.random.lognormal(mu, sigma)

        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help2_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [127]:
class CustomerNorm(Customer):
    """ Customer that has normally distributed expert service"""
    def get_help1_operator1(self):
        duration = random.gauss(mu, sigma)
        while duration < 0:
            duration = random.gauss(mu, sigma)
            
        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [128]:

def customer_generator(env, operator1, operator2):
    """Generate new customer that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator1, operator2)
        

In [129]:
     
def customer_norm_generator(env, operator1, operator2):
    """Generate new customer with normally distributed expert service that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = CustomerNorm('Customer %s' %(i+1), env, operator1, operator2)  
        

In [130]:
def break_generator(env, operator1, operator2):
    """Generate new break for expert."""
    while True:
        if len(queue_w_times_operator1) == CUSTOMER_COUNT:
            break
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        with operator1.request() as req:
            yield req
            print('Operator1 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator1 break done at %g' % (env.now))


        if len(queue_w_times_operator2) == CUSTOMER_COUNT:
            break
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        with operator2.request() as req:
            yield req
            print('Operator2 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator2 break done at %g' % (env.now))


In [131]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator1, operator2))
env.process(break_generator(env, operator1, operator2))
env.run()

Operator1 gives break at 10.1044
Operator1 break done at 13.1044
Operator2 gives break at 18.9602
Operator2 break done at 21.9602
Customer 1 initiated a call at 26.5136
4.356679008537778
8.079502895221648
Customer 2 initiated a call at 31.0856
4.72116963262411
0.7099176347113056
Customer 1 is assigned to a operator2 at 35.2945
Customer 3 initiated a call at 36.7227
3.6364343198456606
8.200319548838715
Customer 4 initiated a call at 38.559
7.536319904163672
8.489145962220032
Customer 1 is done operator2 at 40.7305
Customer 2 A caller hangs up at 41.0184
Customer 3 is assigned to a operator2 at 43.2469
Customer 3 is done operator2 at 50.7199
Customer 4 is assigned to a operator2 at 50.7199
Customer 4 is done operator2 at 53.7679
Customer 5 initiated a call at 60.4655
0.3694895106752971
7.280082284356033
Customer 5 is assigned to a operator1 at 63.0144
Customer 6 initiated a call at 64.8907
9.278803543039864
1.8355848587858192
Customer 5 is done operator1 at 65.6252
Customer 7 initiated a

Customer 375 A caller hangs up at 2156.96
Customer 374 A caller hangs up at 2157.79
Customer 376 initiated a call at 2162.53
2.1285785628792233
1.6355129606729513
Customer 377 initiated a call at 2163.31
8.110746757577585
9.557746757199679
Customer 377 is assigned to a operator2 at 2163.83
Customer 377 is done operator2 at 2167.86
Customer 378 initiated a call at 2177.25
8.084678022112172
0.8065441469252188
Customer 379 initiated a call at 2180.58
3.8686316315491673
3.9781311685575096
Customer 378 A caller hangs up at 2184.95
Customer 379 is assigned to a operator2 at 2188.85
Customer 379 is done operator2 at 2191.1
Customer 380 initiated a call at 2197.57
5.820713647170476
5.544398358451038
Customer 380 is assigned to a operator2 at 2197.86
Customer 381 initiated a call at 2201.62
2.5350610273509897
7.125110297442702
Customer 380 is done operator2 at 2201.78
Customer 358 is done operator1 at 2204.32
Customer 359 is assigned to a operator1 at 2204.32
Customer 382 initiated a call at 22

0.6996394503209147
Customer 614 is done operator1 at 4170.13
Customer 616 is assigned to a operator1 at 4170.13
Customer 678 is done operator2 at 4172.08
Customer 680 is assigned to a operator2 at 4172.14
Customer 681 A caller hangs up at 4172.36
Customer 682 initiated a call at 4172.45
2.294344082824863
2.7220698914573416
Customer 683 initiated a call at 4173.31
8.499971285287147
3.5211609818566747
Customer 680 is done operator2 at 4174.94
Customer 683 is assigned to a operator2 at 4174.94
Customer 683 is done operator2 at 4179.71
Customer 684 initiated a call at 4185.75
2.5254202543621473
2.92313725681134
Customer 685 initiated a call at 4193.81
9.241705602463222
6.782607720462406
Customer 686 initiated a call at 4194.09
6.210622435070798
7.536094909838242
Customer 616 is done operator1 at 4195.02
Customer 619 is assigned to a operator1 at 4195.02
Customer 685 is assigned to a operator2 at 4198.36
Customer 685 is done operator2 at 4202.61
Customer 686 is assigned to a operator2 at 42

Customer 973 A caller drops  without answering. at 5967.71
103
Customer 975 A caller drops  without answering. at 5971.31
102
Customer 942 is done operator1 at 5972.33
Operator1 gives break at 5972.33
Customer 972 A caller hangs up at 5972.84
Operator1 break done at 5975.33
Customer 954 is assigned to a operator1 at 5975.33
Customer 954 is done operator1 at 5976.55
Customer 955 is assigned to a operator1 at 5976.55
Customer 976 initiated a call at 5978.59
5.104903265926328
0.29986997201175036
Customer 976 A caller hangs up at 5979.23
Customer 977 initiated a call at 5979.4
8.300150281723862
5.954522151327602
Customer 977 A caller drops  without answering. at 5981.42
101
Customer 955 is done operator1 at 5987.42
Customer 978 initiated a call at 5987.65
4.110561943138986
1.7943243080042093
Customer 978 is assigned to a operator2 at 5988.82
Customer 979 initiated a call at 5991.46
3.0634904419496323
2.0672376155742755
Customer 978 is done operator2 at 5992.79
Customer 979 is assigned to a

Operator2 gives break at 47676.6
Operator2 break done at 47679.6
Operator1 gives break at 47680.2
Operator1 break done at 47683.2
Operator2 gives break at 47692
Operator2 break done at 47695
Operator1 gives break at 47701.9
Operator1 break done at 47704.9
Operator2 gives break at 47807.8
Operator2 break done at 47810.8
Operator1 gives break at 47837.1
Operator1 break done at 47840.1
Operator2 gives break at 47928.7
Operator2 break done at 47931.7
Operator1 gives break at 48104.4
Operator1 break done at 48107.4
Operator2 gives break at 48131.7
Operator2 break done at 48134.7
Operator1 gives break at 48145.1
Operator1 break done at 48148.1
Operator2 gives break at 48177.4
Operator2 break done at 48180.4
Operator1 gives break at 48202.9
Operator1 break done at 48205.9
Operator2 gives break at 48292.3
Operator2 break done at 48295.3
Operator1 gives break at 48383.2
Operator1 break done at 48386.2
Operator2 gives break at 48495.4
Operator2 break done at 48498.4
Operator1 gives break at 4853

...

Operator1 break done at 141974
Operator2 gives break at 141997
Operator2 break done at 142000
Operator1 gives break at 142053
Operator1 break done at 142056
Operator2 gives break at 142176
Operator2 break done at 142179
Operator1 gives break at 142423
Operator1 break done at 142426
Operator2 gives break at 142452
Operator2 break done at 142455
Operator1 gives break at 142596
Operator1 break done at 142599
Operator2 gives break at 142837
Operator2 break done at 142840
Operator1 gives break at 142873
Operator1 break done at 142876
Operator2 gives break at 142879
Operator2 break done at 142882
Operator1 gives break at 142916
Operator1 break done at 142919
Operator2 gives break at 143081
Operator2 break done at 143084
Operator1 gives break at 143086
Operator1 break done at 143089
Operator2 gives break at 143216
Operator2 break done at 143219
Operator1 gives break at 143225
Operator1 break done at 143228
Operator2 gives break at 143234
Operator2 break done at 143237
Operator1 gives break at

Operator1 gives break at 189615
Operator1 break done at 189618
Operator2 gives break at 189806
Operator2 break done at 189809
Operator1 gives break at 189903
Operator1 break done at 189906
Operator2 gives break at 189943
Operator2 break done at 189946
Operator1 gives break at 190176
Operator1 break done at 190179
Operator2 gives break at 190184
Operator2 break done at 190187
Operator1 gives break at 190229
Operator1 break done at 190232
Operator2 gives break at 190408
Operator2 break done at 190411
Operator1 gives break at 190489
Operator1 break done at 190492
Operator2 gives break at 190568
Operator2 break done at 190571
Operator1 gives break at 190721
Operator1 break done at 190724
Operator2 gives break at 190772
Operator2 break done at 190775
Operator1 gives break at 190821
Operator1 break done at 190824
Operator2 gives break at 190997
Operator2 break done at 191000
Operator1 gives break at 191005
Operator1 break done at 191008
Operator2 gives break at 191048
Operator2 break done at

Operator1 gives break at 239023
Operator1 break done at 239026
Operator2 gives break at 239027
Operator2 break done at 239030
Operator1 gives break at 239072
Operator1 break done at 239075
Operator2 gives break at 239078
Operator2 break done at 239081
Operator1 gives break at 239096
Operator1 break done at 239099
Operator2 gives break at 239264
Operator2 break done at 239267
Operator1 gives break at 239380
Operator1 break done at 239383
Operator2 gives break at 239413
Operator2 break done at 239416
Operator1 gives break at 239496
Operator1 break done at 239499
Operator2 gives break at 239684
Operator2 break done at 239687
Operator1 gives break at 239800
Operator1 break done at 239803
Operator2 gives break at 239911
Operator2 break done at 239914
Operator1 gives break at 239978
Operator1 break done at 239981
Operator2 gives break at 240034
Operator2 break done at 240037
Operator1 gives break at 240190
Operator1 break done at 240193
Operator2 gives break at 240450
Operator2 break done at

Operator2 break done at 286720
Operator1 gives break at 286826
Operator1 break done at 286829
Operator2 gives break at 286890
Operator2 break done at 286893
Operator1 gives break at 286920
Operator1 break done at 286923
Operator2 gives break at 286943
Operator2 break done at 286946
Operator1 gives break at 286987
Operator1 break done at 286990
Operator2 gives break at 287036
Operator2 break done at 287039
Operator1 gives break at 287088
Operator1 break done at 287091
Operator2 gives break at 287101
Operator2 break done at 287104
Operator1 gives break at 287130
Operator1 break done at 287133
Operator2 gives break at 287226
Operator2 break done at 287229
Operator1 gives break at 287288
Operator1 break done at 287291
Operator2 gives break at 287351
Operator2 break done at 287354
Operator1 gives break at 287567
Operator1 break done at 287570
Operator2 gives break at 287593
Operator2 break done at 287596
Operator1 gives break at 287614
Operator1 break done at 287617
Operator2 gives break at

KeyboardInterrupt: 